In [17]:
#import necessary modules

import pandas as pd
from pymongo import MongoClient
from datetime import datetime



In [18]:

# MongoDB Connection
client = MongoClient("mongodb://localhost:27017/")
db = client['air_quality_db']  # Create or connect to the database
collection = db['dhaka_pm25_data']  # Create or connect to the collection

# Load your cleansed dataset
csv_file = r"C:\Users\Shafi Hussain\OneDrive\Desktop\DSCI 105\FINAL PROJECT DATASETS\cleaned_airqualityDhaka_dataset.csv"
data = pd.read_csv(csv_file)

# Convert 'DateTime' column to datetime format for MongoDB
data['DateTime'] = pd.to_datetime(data['DateTime'])

# Convert DataFrame to JSON records
records = data.to_dict(orient='records')

# Insert data into MongoDB
collection.insert_many(records)

print("Data successfully inserted into MongoDB!")


Data successfully inserted into MongoDB!


In [19]:
#Create single-field index on datetime field

collection.create_index("DateTime")

#creates a compound index on year, month and day fields in ascending order

collection.create_index([("Year", 1), ("Month", 1), ("Day", 1)])

#create a single-field index on AQI category 

collection.create_index("AQICategory")


'AQICategory_1'

In [20]:
#retreieves all the records for January. Alter Month/Year as preferred

query = {"Year": 2023, "Month": 1}
results = collection.find(query)
for record in results:
    print(record)


{'_id': ObjectId('6760dba0544782aad61c6606'), 'DateTime': datetime.datetime(2023, 1, 1, 0, 0), 'Site': 'Dhaka', 'Parameter': 'PM2.5 - Principal', 'Year': 2023, 'Month': 1, 'Day': 1, 'Hour': 0, 'NowCast Conc.': 255.2, 'AQI': 306.0, 'AQI Category': 'Hazardous', 'Raw Conc.': 249.0, 'Conc. Unit': 'UG/M3', 'Duration': '1 Hr', 'QC Name': 'Valid', 'AQI_7Day_RollingAvg': 291.92857142857144}
{'_id': ObjectId('6760dba0544782aad61c6607'), 'DateTime': datetime.datetime(2023, 1, 1, 1, 0), 'Site': 'Dhaka', 'Parameter': 'PM2.5 - Principal', 'Year': 2023, 'Month': 1, 'Day': 1, 'Hour': 1, 'NowCast Conc.': 297.6, 'AQI': 348.0, 'AQI Category': 'Hazardous', 'Raw Conc.': 340.0, 'Conc. Unit': 'UG/M3', 'Duration': '1 Hr', 'QC Name': 'Valid', 'AQI_7Day_RollingAvg': 291.57738095238096}
{'_id': ObjectId('6760dba0544782aad61c6608'), 'DateTime': datetime.datetime(2023, 1, 1, 2, 0), 'Site': 'Dhaka', 'Parameter': 'PM2.5 - Principal', 'Year': 2023, 'Month': 1, 'Day': 1, 'Hour': 2, 'NowCast Conc.': 254.8, 'AQI': 305.

In [21]:
#get all the records for any AQI category

query = {"AQICategory": "Hazardous"}
results = collection.find(query)
for record in results:
    print(record)


In [22]:
#Finds records for a specific datetime range

query = {
    "DateTime": {
        "$gte": datetime(2023, 12, 1),
        "$lte": datetime(2023, 12, 31)
    }
}
results = collection.find(query)
for record in results:
    print(record)


{'_id': ObjectId('6760dba0544782aad61c8556'), 'DateTime': datetime.datetime(2023, 12, 1, 0, 0), 'Site': 'Dhaka', 'Parameter': 'PM2.5 - Principal', 'Year': 2023, 'Month': 12, 'Day': 1, 'Hour': 0, 'NowCast Conc.': 149.5, 'AQI': 200.0, 'AQI Category': 'Unhealthy', 'Raw Conc.': 144.0, 'Conc. Unit': 'UG/M3', 'Duration': '1 Hr', 'QC Name': 'Valid', 'AQI_7Day_RollingAvg': 199.375}
{'_id': ObjectId('6760dd1a544782aad61d74f7'), 'DateTime': datetime.datetime(2023, 12, 1, 0, 0), 'Site': 'Dhaka', 'Parameter': 'PM2.5 - Principal', 'Year': 2023, 'Month': 12, 'Day': 1, 'Hour': 0, 'NowCast Conc.': 149.5, 'AQI': 200.0, 'AQI Category': 'Unhealthy', 'Raw Conc.': 144.0, 'Conc. Unit': 'UG/M3', 'Duration': '1 Hr', 'QC Name': 'Valid', 'AQI_7Day_RollingAvg': 199.375}
{'_id': ObjectId('6760dd22544782aad61e6498'), 'DateTime': datetime.datetime(2023, 12, 1, 0, 0), 'Site': 'Dhaka', 'Parameter': 'PM2.5 - Principal', 'Year': 2023, 'Month': 12, 'Day': 1, 'Hour': 0, 'NowCast Conc.': 149.5, 'AQI': 200.0, 'AQI Category

In [23]:
#aggregate average AQI by month

pipeline = [
    {"$group": {
        "_id": {"Year": "$Year", "Month": "$Month"},
        "Average_AQI": {"$avg": "$AQI"}
    }},
    {"$sort": {"_id.Year": 1, "_id.Month": 1}}
]

results = collection.aggregate(pipeline)
for result in results:
    print(result)


{'_id': {'Year': 2017, 'Month': 1}, 'Average_AQI': 248.03762912160326}
{'_id': {'Year': 2017, 'Month': 2}, 'Average_AQI': 227.72212616010026}
{'_id': {'Year': 2017, 'Month': 3}, 'Average_AQI': 167.78745121930794}
{'_id': {'Year': 2017, 'Month': 4}, 'Average_AQI': 138.4611111111111}
{'_id': {'Year': 2017, 'Month': 5}, 'Average_AQI': 125.83479758465737}
{'_id': {'Year': 2017, 'Month': 6}, 'Average_AQI': 91.49723557692307}
{'_id': {'Year': 2017, 'Month': 7}, 'Average_AQI': 87.98477224062624}
{'_id': {'Year': 2017, 'Month': 8}, 'Average_AQI': 94.76478494623656}
{'_id': {'Year': 2017, 'Month': 9}, 'Average_AQI': 99.5546080932875}
{'_id': {'Year': 2017, 'Month': 10}, 'Average_AQI': 117.69527883851494}
{'_id': {'Year': 2017, 'Month': 11}, 'Average_AQI': 164.00972222222222}
{'_id': {'Year': 2017, 'Month': 12}, 'Average_AQI': 195.38172043010752}
{'_id': {'Year': 2018, 'Month': 1}, 'Average_AQI': 255.28431411872228}
{'_id': {'Year': 2018, 'Month': 2}, 'Average_AQI': 221.9900173611111}
{'_id': {'

In [24]:
#Count records by AQI category

pipeline = [
    {"$group": {"_id": "$AQICategory", "Count": {"$sum": 1}}},
    {"$sort": {"Count": -1}}
]

results = collection.aggregate(pipeline)
for result in results:
    print(result)


{'_id': None, 'Count': 245376}
